<a href="https://colab.research.google.com/github/MrSottaceti/Portfolio_optimization/blob/main/Batmmaan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from fredapi import Fred
import cvxpy as cp
import matplotlib.pyplot as plt

Tramite la libreria yfinance scarichiamo i dati delle 7 azioni che comporranno il portafoglio.

In [12]:
#Impostiamo il vettore tickers dei BATMMAAN
tickers = ["AVGO", "AAPL", "TSLA", "MSFT", "META", "AMZN", "GOOGL", "NVDA"]

#Scarichiamo i dati delle azioni con yfinance
start_date='2020-01-01'
end_date='2025-04-18'
data = yf.download(tickers, start=start_date, end=end_date)

[*********************100%***********************]  8 of 8 completed


Usiamo i dati scaricati per calcolare i rendimenti percentuali. Su questi, poi, estraiamo la matrice covarianze. Questa esprime varianze e covarianze su base giornaliera quindi la moltiplichiamo per 252.

Si rende necessario indicizzare la matrice covarianze con .loc per rispettare l'ordine di inserimento dei ticker ("BATMMAAN")

In [31]:
#Prendiamo i prezzi di chiusura e li usiamo per calcolare i rendimenti giornalieri in percentuale
close_prices = data['Close']
returns = close_prices.pct_change().dropna() #Serve .dropna() a quanto pare

#se serve, var_returns = returns.var() ma usiamo la matrice covarianze quindi non dovrebbe servire
#Calcoliamo la matrice covarianze, .loc[...] serve a indicizzare gli elementi
#nell'ordine indicato all'inizio nel vettore tickers. Altrimenti uscirebbe in ordine alfabetico
cov_matrix = returns.cov()*252
cov_matrix = cov_matrix.loc[tickers, tickers] #print(cov_matrix)

#print(cov_matrix)

Ticker      AVGO      AAPL      TSLA      MSFT      META      AMZN     GOOGL  \
Ticker                                                                         
AVGO    0.193544  0.080424  0.139061  0.081096  0.094848  0.080888  0.077543   
AAPL    0.080424  0.107581  0.112095  0.074368  0.084582  0.071735  0.069749   
TSLA    0.139061  0.112095  0.465680  0.096720  0.110484  0.113061  0.095571   
MSFT    0.081096  0.074368  0.096720  0.093738  0.086268  0.076209  0.074498   
META    0.094848  0.084582  0.110484  0.086268  0.203240  0.102515  0.095537   
AMZN    0.080888  0.071735  0.113061  0.076209  0.102515  0.131757  0.078464   
GOOGL   0.077543  0.069749  0.095571  0.074498  0.095537  0.078464  0.107818   
NVDA    0.165714  0.108243  0.181635  0.116007  0.135863  0.118408  0.108757   

Ticker      NVDA  
Ticker            
AVGO    0.165714  
AAPL    0.108243  
TSLA    0.181635  
MSFT    0.116007  
META    0.135863  
AMZN    0.118408  
GOOGL   0.108757  
NVDA    0.305619  


Per questa analisi diamo per buono il CAPM (anche se non lo è). Quindi ci andiamo a calcolare il beta e otteniamo il rendimento risk free.

In [23]:
#Prima di iniziare con l'ottimizzazione dobbiamo prima stimare i rendimenti attesi.
#Utilizziamo il CAPM. Per prima cosa ci serve anche il VETTORE BETA (covarianza tra titolo e mercato(SP500) su varianza del mercato)

ticker_mkt = "^GSPC" #ticker dell's&p 500
sp500_data = yf.download(ticker_mkt, start=start_date, end=end_date)
sp500_close = sp500_data['Close']
returns_mkt = sp500_close.pct_change().dropna()

returns = close_prices.resample('ME').last().pct_change().dropna()
returns_mkt = sp500_close.resample('ME').last().pct_change().dropna()

#PER OTTENERE UN ARRAY DEI  BETA PRIMA CREIAMO IL DIZIONARIO betas_dict e poi lo trasformiamo in ARRAY
betas_dict = {}
for ticker in tickers:  #Estrai i rendimenti dell'azione e del mercato
    y = returns[ticker]  # Rendimenti dell'azione
    X = returns_mkt  # Rendimenti del mercato (S&P 500)
    df = pd.concat([y, X], axis=1).dropna() #Ricalcola y e X senza NaN
    y = df.iloc[:, 0]
    X = df.iloc[:, 1]
    #Calcola beta
    beta = np.cov(y, X)[0, 1] / np.var(X, ddof=1)  # Cov. tra azione e mercato normalizzata (NB!!) per la var. CAMPIONARIA (DDOF=1) del mercato
    betas_dict[ticker] = beta

#se hai bisogno di stampare i beta elencati per ciascuna azione, utilizza:
for ticker, beta in betas_dict.items():
   print(f"Beta di {ticker}: {beta:.4f}")

betas = np.array(list(betas_dict.values())).reshape(-1, 1)  # (8x1)
#print(betas) #se vuoi vedere l'array dei beta.

[*********************100%***********************]  1 of 1 completed


Beta di AVGO: 1.0532
Beta di AAPL: 1.2297
Beta di TSLA: 2.3623
Beta di MSFT: 0.8922
Beta di META: 1.2640
Beta di AMZN: 1.1942
Beta di GOOGL: 1.0132
Beta di NVDA: 1.6408


In alternativa il vettore BETA 5Y mensile possiamo ottenerlo così:

In [27]:
beta_data = yf.download(tickers + [ticker_mkt], period="5y", interval="1mo")["Close"]
#head(beta_data)

#Rendimenti mensili
returns_2 = beta_data.pct_change().dropna()

stock_returns = returns_2[tickers]
market_returns = returns_2[ticker_mkt]

betas_2 = {}
for ticker in tickers:
    covariance = stock_returns[ticker].cov(market_returns)
    market_variance = market_returns.var()
    betas_2[ticker] = covariance / market_variance

# Stampa risultati
print("Beta dei titoli (5Y, mensile):")
print(pd.Series(betas_2).round(2))

[*********************100%***********************]  9 of 9 completed

Beta dei titoli (5Y, mensile):
AVGO     1.06
AAPL     1.29
TSLA     2.43
MSFT     0.98
META     1.26
AMZN     1.33
GOOGL    0.99
NVDA     2.07
dtype: float64


Ma andiamo avanti. Ci serve il tasso risk free e lo otteniamo con fredapi.

Per comodità lascio la mia key a disposizione ma è buona cosa usarne una propria.

In [29]:
#usiamo l'api della Fred (from fredapi import Fred) per avere il tasso a 10 anni Treasury
#inseriamo l'API Key FRED
fred = Fred(api_key="f0ee647a3e3b4016c1fdf3530dfb8047")
tbill_10y_series = fred.get_series("DGS10")

#otteniamo l'ultimo valore disponibile del tasso treasury a 10 anni (lo useremo come risk free rate)
t_10 = tbill_10y_series.iloc[-1] / 100  #con .iloc[-1] otteniamo l'ultimo valore disponibile e con "/100" lo convertiamo da percentuale a decimale
rf = t_10
print(f"Tasso Treasury 10Y: {tbill_10y:.4%}")

Tasso Treasury 10Y: 4.3400%


Ora riempiamo un vettore di risk free rate ottenuto. Questo vettore ci serve per ottenere gli expected returns dei singoli titoli tramite modello capm.
\begin{equation}
\underbrace{
\begin{bmatrix}
E[r_1] \\
E[r_2] \\
\vdots \\
E[r_n]
\end{bmatrix}
}_{\mathbf{E}[r]}
=
\underbrace{
\begin{bmatrix}
r_{f,1} \\
r_{f,2} \\
\vdots \\
r_{f,n}
\end{bmatrix}
}_{\mathbf{r_f}}
+
\underbrace{
\begin{bmatrix}
\beta_1 \\
\beta_2 \\
\vdots \\
\beta_n
\end{bmatrix}
}_{\boldsymbol{\beta}}
\cdot
\underbrace{
\left(E[r_m] - r_f\right)
}_{\mathrm{ERP}}
\end{equation}

In [ ]:
num_assets = len(tickers)  # Conta quanti sono gli asset
rf_vett = np.full((num_assets, 1), rf) #Crea il vettore con il tasso risk-free ripetuto N volte

expected_return_sp500 = 0.1 #ATTENZIONE dato esogeno: GOLDMAN SACHS stima +10% nel 2025
eq_risk_pr = expected_return_sp500 - rf

expected_return_batmmaan_capm = rf_vett + betas * eq_risk_pr
#print(expected_return_batmmaan_capm)

#Appiattiamo (collassiamo) il vettore dei rendimenti attesi in un array 1D
expected_return = expected_return_batmmaan_capm.flatten()

Definiamo portfolio_return e portfolio_variance. Introduciamo i primi constraints fondamentali e aggiungiamo il vincolo di un certo return (qui 12%) e upper e lower bound.
\begin{equation}
\sigma_p^2 =
\begin{bmatrix}
w_1 & w_2 & \cdots & w_n
\end{bmatrix}
\begin{bmatrix}
\sigma_{11} & \sigma_{12} & \cdots & \sigma_{1n} \\
\sigma_{21} & \sigma_{22} & \cdots & \sigma_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
\sigma_{n1} & \sigma_{n2} & \cdots & \sigma_{nn}
\end{bmatrix}
\begin{bmatrix}
w_1 \\
w_2 \\
\vdots \\
w_n
\end{bmatrix}
\end{equation}

constraints.append(...) aggiunge i vincoli all'ottimizzatore



In [ ]:
#creiamo vettore pesi w, per lanciare ottimizzazione. len() conta numero asset che corrisponderà alla dimensione del vettore
num_assets = len(expected_return)
w = cp.Variable(num_assets)

portfolio_return = expected_return @ w  #Calcoliamo il rendimento atteso del portafoglio

# Calcoliamo la varianza del portafoglio usando la matrice di covarianza
# Convertiamo la matrice di covarianza in array numpy se necessario
cov_array = cov_matrix.values
portfolio_variance = cp.quad_form(w, cov_array) #w'*sigma*w

#Definiamo il vettore dei constraints con i vincoli "base" (NO SHORT; 100% CAP. INVEST.)
constraints = [
    cp.sum(w) == 1,
    w >= 0
]
#A questo vettore possiamo aggiungere tutti i constraints che vogliamo (con .append)
target_return = 0.12  # ad esempio, il 12%
constraints.append(portfolio_return >= target_return) #>= oppure ==

#aggiungiamo anche un vincolo di peso massimo e minimo
lower_bound = 0.05
upper_bound = 0.30
constraints.append(w >= lower_bound)
constraints.append(w <= upper_bound)

L'ottimizzatore MINIMIZZA la varianza dati i constraints. NB: l'oggetto portfolio_variance contiene il vettore w, sul quale l'ottimizzatore interviene.
problem.solve()

In [ ]:
# Definiamo il problema: minimizzazione della varianza
problem = cp.Problem(cp.Minimize(portfolio_variance), constraints)
problem.solve() #risolviamo il problema

optimal_weights = w.value #Estraiamo i pesi ottimali

Così sintetizziamo l'output dell'ottimizzatore.

In [ ]:
print("Pesi ottimali:", optimal_weights)
print("Rendimento atteso del portafoglio:", portfolio_return.value)
print("Standard dev del portafoglio:", np.sqrt(portfolio_variance.value)*100)

optimal_weights_percent = np.round(optimal_weights * 100, 2)
print(optimal_weights_percent)

ticker_weights = dict(zip(tickers, optimal_weights_percent))

for ticker, weight in ticker_weights.items():
    print(f"{ticker}: {weight:.2f}%")  #così stampi i pesi ottimali in lista


In [ ]:
#PER LA CLASSICA PIE CHART:
#imposto i colori della torta (nvidia verde, meta azzurra, etc....)
colors = ['#CC092F', '#A2AAAD', '#e82127', '#00A4EF', '#0080FB', '#ff9900', '#F4B400', '#76b900']

plt.figure(figsize=(8, 8))
plt.pie(optimal_weights_percent, labels=["Broadcom", "Apple", "Tesla", "Microsoft", "Meta", "Amazon", "Alphabet", "Nvidia"], autopct='%1.1f%%', startangle=140, colors=colors, counterclock=False)
plt.title("BATMMAAN OPTIMIZED ALLOCATION")
plt.show()